In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
#from pydub import AudioSegment
#from pydub.playback import play

from sklearn.naive_bayes import GaussianNB,MultinomialNB, BernoulliNB, ComplementNB, CategoricalNB
from sklearn.neighbors import KNeighborsClassifier

In [7]:
df = pd.read_csv("features_3_sec.csv")
labels=df['label']
num=0
one_hot_encode={}
labels_encoded=[]

for i in labels:
    if i not in one_hot_encode.keys():
        one_hot_encode[i]=num
        labels_encoded.append(num)
        num+=1
    else:
        labels_encoded.append(one_hot_encode[i])
df['labels_encoded']=labels_encoded
print(one_hot_encode)
df


{'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}


,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label,labels_encoded
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues,0
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues,0
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues,0
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues,0
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,rock.00099.5.wav,66149,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,...,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock,9
9986,rock.00099.6.wav,66149,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,...,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock,9
9987,rock.00099.7.wav,66149,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,...,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock,9
9988,rock.00099.8.wav,66149,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,...,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock,9


In [3]:
#song = AudioSegment.from_wav("samples/classical.00013.wav")

#play(song)

In [8]:
X = df.copy()
X.drop(['filename', 'label','length','labels_encoded'],axis=1,inplace=True)
display(X)
Y=pd.DataFrame(df['labels_encoded'])
display(Y)

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,1.080790e+06,...,-2.853603,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767
1,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,6.722448e+05,...,4.074709,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081
2,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,7.907127e+05,...,4.806280,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122
3,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,9.216524e+05,...,-1.359111,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678
4,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,6.102111e+05,...,2.092937,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,3015.559458,8.479527e+05,...,5.773784,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969
9986,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,3746.694524,1.170890e+06,...,2.074155,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033
9987,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,2442.362154,2.602871e+06,...,-1.005473,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990
9988,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,4313.266226,4.968878e+05,...,4.123402,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750


,labels_encoded
0,0
1,0
2,0
3,0
4,0
...,...
9985,9
9986,9
9987,9
9988,9


In [11]:
print(X.columns)
len(X.columns)

Index(['chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var',
       'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo',
       'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var'],
      dtype='object')


57

The features used for training

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=int(0.1*len(X)), random_state=1234)
print(x_train.shape)
y_test=np.array(y_test)
y_train=np.array(y_train)
y_test=y_test[:,0]
y_train=y_train[:,0]

(900, 57)


Gaussian Naive Bayes

In [6]:
param_grid = {'var_smoothing': [0,1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]}
 
grid = GridSearchCV(GaussianNB(), param_grid, refit = True, verbose = 3)

grid.fit(x_train, y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
[CV 1/5] END ...................var_smoothing=0;, score=0.511 total time=   0.0s
[CV 2/5] END ...................var_smoothing=0;, score=0.517 total time=   0.0s
[CV 3/5] END ...................var_smoothing=0;, score=0.539 total time=   0.0s
[CV 4/5] END ...................var_smoothing=0;, score=0.594 total time=   0.0s
[CV 5/5] END ...................var_smoothing=0;, score=0.539 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-10;, score=0.417 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-10;, score=0.444 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-10;, score=0.489 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-10;, score=0.489 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-10;, score=0.461 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-09;, score=0.433 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-09;

GridSearchCV(estimator=GaussianNB(),
             param_grid={'var_smoothing': [0, 1e-10, 1e-09, 1e-08, 1e-07, 1e-06,
                                           1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=3)

In [7]:
grid_predictions = grid.predict(x_test)
print("Best param:",grid.best_params_)
print("Best estimator:",grid.best_estimator_)
print("Classification Report:",classification_report(y_test, grid_predictions))

Best param: {'var_smoothing': 0}
Best estimator: GaussianNB(var_smoothing=0)
Classification Report:               precision    recall  f1-score   support

           0       0.60      0.43      0.50         7
           1       0.71      1.00      0.83        12
           2       0.40      0.57      0.47         7
           3       0.00      0.00      0.00         9
           4       0.56      0.38      0.45        13
           5       0.57      0.44      0.50         9
           6       0.57      0.80      0.67        10
           7       0.64      0.69      0.67        13
           8       0.42      0.42      0.42        12
           9       0.20      0.12      0.15         8

    accuracy                           0.51       100
   macro avg       0.47      0.49      0.47       100
weighted avg       0.49      0.51      0.49       100



In [8]:
#For BernoulliNB
param_grid = {'alpha': [0,1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1,1,10,100,1000],'fit_prior':[True,False]}
 
grid = GridSearchCV(BernoulliNB(), param_grid, refit = True, verbose = 3)

grid.fit(x_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5] END ...........alpha=0, fit_prior=True;, score=0.283 total time=   0.0s
[CV 2/5] END ...........alpha=0, fit_prior=True;, score=0.339 total time=   0.0s
[CV 3/5] END ...........alpha=0, fit_prior=True;, score=0.350 total time=   0.0s
[CV 4/5] END ...........alpha=0, fit_prior=True;, score=0.333 total time=   0.0s
[CV 5/5] END ...........alpha=0, fit_prior=True;, score=0.289 total time=   0.0s
[CV 1/5] END ..........alpha=0, fit_prior=False;, score=0.283 total time=   0.0s
[CV 2/5] END ..........alpha=0, fit_prior=False;, score=0.339 total time=   0.0s
[CV 3/5] END ..........alpha=0, fit_prior=False;, score=0.350 total time=   0.0s
[CV 4/5] END ..........alpha=0, fit_prior=False;, score=0.333 total time=   0.0s
[CV 5/5] END ..........alpha=0, fit_prior=False;, score=0.278 total time=   0.0s
[CV 1/5] END .......alpha=1e-10, fit_prior=True;, score=0.283 total time=   0.0s
[CV 2/5] END .......alpha=1e-10, fit_prior=True

/Users/kalp_vyas/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/kalp_vyas/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/kalp_vyas/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/kalp_vyas/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/kalp_vyas/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/kalp_vyas/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes

[CV 3/5] END ......alpha=1e-06, fit_prior=False;, score=0.350 total time=   0.0s
[CV 4/5] END ......alpha=1e-06, fit_prior=False;, score=0.333 total time=   0.0s
[CV 5/5] END ......alpha=1e-06, fit_prior=False;, score=0.278 total time=   0.0s
[CV 1/5] END .......alpha=1e-05, fit_prior=True;, score=0.283 total time=   0.0s
[CV 2/5] END .......alpha=1e-05, fit_prior=True;, score=0.339 total time=   0.0s
[CV 3/5] END .......alpha=1e-05, fit_prior=True;, score=0.350 total time=   0.0s
[CV 4/5] END .......alpha=1e-05, fit_prior=True;, score=0.333 total time=   0.0s
[CV 5/5] END .......alpha=1e-05, fit_prior=True;, score=0.289 total time=   0.0s
[CV 1/5] END ......alpha=1e-05, fit_prior=False;, score=0.283 total time=   0.0s
[CV 2/5] END ......alpha=1e-05, fit_prior=False;, score=0.339 total time=   0.0s
[CV 3/5] END ......alpha=1e-05, fit_prior=False;, score=0.350 total time=   0.0s
[CV 4/5] END ......alpha=1e-05, fit_prior=False;, score=0.333 total time=   0.0s
[CV 5/5] END ......alpha=1e-

/Users/kalp_vyas/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


GridSearchCV(estimator=BernoulliNB(),
             param_grid={'alpha': [0, 1e-10, 1e-09, 1e-08, 1e-07, 1e-06, 1e-05,
                                   0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'fit_prior': [True, False]},
             verbose=3)

In [9]:
grid_predictions = grid.predict(x_test)
print("Best param:",grid.best_params_)
print("Best estimator:",grid.best_estimator_)
print("Classification Report:",classification_report(y_test, grid_predictions))

Best param: {'alpha': 0, 'fit_prior': True}
Best estimator: BernoulliNB(alpha=0)
Classification Report:               precision    recall  f1-score   support

           0       0.12      0.14      0.13         7
           1       0.43      0.75      0.55        12
           2       0.10      0.14      0.12         7
           3       0.00      0.00      0.00         9
           4       0.00      0.00      0.00        13
           5       0.00      0.00      0.00         9
           6       0.31      0.80      0.44        10
           7       0.53      0.69      0.60        13
           8       0.33      0.25      0.29        12
           9       0.00      0.00      0.00         8

    accuracy                           0.31       100
   macro avg       0.18      0.28      0.21       100
weighted avg       0.21      0.31      0.24       100



KNN algorithm

In [12]:
param_grid = {'n_neighbors': [x for x in range(1,100)],'weights':['uniform','distance'],'algorithm':['auto','ball_tree','kd_tree','brute'],'p':[1,2]}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3)

grid.fit(x_train, y_train)

Fitting 5 folds for each of 1584 candidates, totalling 7920 fits
[CV 1/5] END algorithm=auto, n_neighbors=1, p=1, weights=uniform;, score=0.300 total time=   0.0s
[CV 2/5] END algorithm=auto, n_neighbors=1, p=1, weights=uniform;, score=0.261 total time=   0.0s
[CV 3/5] END algorithm=auto, n_neighbors=1, p=1, weights=uniform;, score=0.250 total time=   0.0s
[CV 4/5] END algorithm=auto, n_neighbors=1, p=1, weights=uniform;, score=0.322 total time=   0.0s
[CV 5/5] END algorithm=auto, n_neighbors=1, p=1, weights=uniform;, score=0.294 total time=   0.0s
[CV 1/5] END algorithm=auto, n_neighbors=1, p=1, weights=distance;, score=0.300 total time=   0.0s
[CV 2/5] END algorithm=auto, n_neighbors=1, p=1, weights=distance;, score=0.261 total time=   0.0s
[CV 3/5] END algorithm=auto, n_neighbors=1, p=1, weights=distance;, score=0.250 total time=   0.0s
[CV 4/5] END algorithm=auto, n_neighbors=1, p=1, weights=distance;, score=0.322 total time=   0.0s
[CV 5/5] END algorithm=auto, n_neighbors=1, p=1, 

[CV 2/5] END algorithm=auto, n_neighbors=5, p=1, weights=distance;, score=0.283 total time=   0.0s
[CV 3/5] END algorithm=auto, n_neighbors=5, p=1, weights=distance;, score=0.244 total time=   0.0s
[CV 4/5] END algorithm=auto, n_neighbors=5, p=1, weights=distance;, score=0.328 total time=   0.0s
[CV 5/5] END algorithm=auto, n_neighbors=5, p=1, weights=distance;, score=0.267 total time=   0.0s
[CV 1/5] END algorithm=auto, n_neighbors=5, p=2, weights=uniform;, score=0.261 total time=   0.0s
[CV 2/5] END algorithm=auto, n_neighbors=5, p=2, weights=uniform;, score=0.256 total time=   0.0s
[CV 3/5] END algorithm=auto, n_neighbors=5, p=2, weights=uniform;, score=0.194 total time=   0.0s
[CV 4/5] END algorithm=auto, n_neighbors=5, p=2, weights=uniform;, score=0.283 total time=   0.0s
[CV 5/5] END algorithm=auto, n_neighbors=5, p=2, weights=uniform;, score=0.256 total time=   0.0s
[CV 1/5] END algorithm=auto, n_neighbors=5, p=2, weights=distance;, score=0.294 total time=   0.0s
[CV 2/5] END al

[CV 3/5] END algorithm=auto, n_neighbors=10, p=1, weights=distance;, score=0.267 total time=   0.0s
[CV 4/5] END algorithm=auto, n_neighbors=10, p=1, weights=distance;, score=0.344 total time=   0.0s
[CV 5/5] END algorithm=auto, n_neighbors=10, p=1, weights=distance;, score=0.306 total time=   0.0s
[CV 1/5] END algorithm=auto, n_neighbors=10, p=2, weights=uniform;, score=0.283 total time=   0.0s
[CV 2/5] END algorithm=auto, n_neighbors=10, p=2, weights=uniform;, score=0.272 total time=   0.0s
[CV 3/5] END algorithm=auto, n_neighbors=10, p=2, weights=uniform;, score=0.200 total time=   0.0s
[CV 4/5] END algorithm=auto, n_neighbors=10, p=2, weights=uniform;, score=0.322 total time=   0.0s
[CV 5/5] END algorithm=auto, n_neighbors=10, p=2, weights=uniform;, score=0.328 total time=   0.0s
[CV 1/5] END algorithm=auto, n_neighbors=10, p=2, weights=distance;, score=0.300 total time=   0.0s
[CV 2/5] END algorithm=auto, n_neighbors=10, p=2, weights=distance;, score=0.278 total time=   0.0s
[CV 3

[CV 4/5] END algorithm=auto, n_neighbors=15, p=2, weights=uniform;, score=0.322 total time=   0.0s
[CV 5/5] END algorithm=auto, n_neighbors=15, p=2, weights=uniform;, score=0.333 total time=   0.0s
[CV 1/5] END algorithm=auto, n_neighbors=15, p=2, weights=distance;, score=0.294 total time=   0.0s
[CV 2/5] END algorithm=auto, n_neighbors=15, p=2, weights=distance;, score=0.306 total time=   0.0s
[CV 3/5] END algorithm=auto, n_neighbors=15, p=2, weights=distance;, score=0.267 total time=   0.0s
[CV 4/5] END algorithm=auto, n_neighbors=15, p=2, weights=distance;, score=0.344 total time=   0.0s
[CV 5/5] END algorithm=auto, n_neighbors=15, p=2, weights=distance;, score=0.322 total time=   0.0s
[CV 1/5] END algorithm=auto, n_neighbors=16, p=1, weights=uniform;, score=0.272 total time=   0.0s
[CV 2/5] END algorithm=auto, n_neighbors=16, p=1, weights=uniform;, score=0.278 total time=   0.0s
[CV 3/5] END algorithm=auto, n_neighbors=16, p=1, weights=uniform;, score=0.267 total time=   0.0s
[CV 4

[CV 2/5] END algorithm=auto, n_neighbors=20, p=1, weights=distance;, score=0.317 total time=   0.0s
[CV 3/5] END algorithm=auto, n_neighbors=20, p=1, weights=distance;, score=0.278 total time=   0.0s
[CV 4/5] END algorithm=auto, n_neighbors=20, p=1, weights=distance;, score=0.356 total time=   0.0s
[CV 5/5] END algorithm=auto, n_neighbors=20, p=1, weights=distance;, score=0.344 total time=   0.0s
[CV 1/5] END algorithm=auto, n_neighbors=20, p=2, weights=uniform;, score=0.272 total time=   0.0s
[CV 2/5] END algorithm=auto, n_neighbors=20, p=2, weights=uniform;, score=0.267 total time=   0.0s
[CV 3/5] END algorithm=auto, n_neighbors=20, p=2, weights=uniform;, score=0.289 total time=   0.0s
[CV 4/5] END algorithm=auto, n_neighbors=20, p=2, weights=uniform;, score=0.333 total time=   0.0s
[CV 5/5] END algorithm=auto, n_neighbors=20, p=2, weights=uniform;, score=0.300 total time=   0.0s
[CV 1/5] END algorithm=auto, n_neighbors=20, p=2, weights=distance;, score=0.311 total time=   0.0s
[CV 2

...

[CV 1/5] END algorithm=brute, n_neighbors=86, p=1, weights=uniform;, score=0.256 total time=   0.0s
[CV 2/5] END algorithm=brute, n_neighbors=86, p=1, weights=uniform;, score=0.267 total time=   0.0s
[CV 3/5] END algorithm=brute, n_neighbors=86, p=1, weights=uniform;, score=0.306 total time=   0.0s
[CV 4/5] END algorithm=brute, n_neighbors=86, p=1, weights=uniform;, score=0.328 total time=   0.0s
[CV 5/5] END algorithm=brute, n_neighbors=86, p=1, weights=uniform;, score=0.261 total time=   0.0s
[CV 1/5] END algorithm=brute, n_neighbors=86, p=1, weights=distance;, score=0.267 total time=   0.0s
[CV 2/5] END algorithm=brute, n_neighbors=86, p=1, weights=distance;, score=0.300 total time=   0.0s
[CV 3/5] END algorithm=brute, n_neighbors=86, p=1, weights=distance;, score=0.311 total time=   0.0s
[CV 4/5] END algorithm=brute, n_neighbors=86, p=1, weights=distance;, score=0.356 total time=   0.0s
[CV 5/5] END algorithm=brute, n_neighbors=86, p=1, weights=distance;, score=0.322 total time=   

[CV 3/5] END algorithm=brute, n_neighbors=90, p=1, weights=distance;, score=0.300 total time=   0.0s
[CV 4/5] END algorithm=brute, n_neighbors=90, p=1, weights=distance;, score=0.356 total time=   0.0s
[CV 5/5] END algorithm=brute, n_neighbors=90, p=1, weights=distance;, score=0.328 total time=   0.0s
[CV 1/5] END algorithm=brute, n_neighbors=90, p=2, weights=uniform;, score=0.239 total time=   0.0s
[CV 2/5] END algorithm=brute, n_neighbors=90, p=2, weights=uniform;, score=0.233 total time=   0.0s
[CV 3/5] END algorithm=brute, n_neighbors=90, p=2, weights=uniform;, score=0.289 total time=   0.0s
[CV 4/5] END algorithm=brute, n_neighbors=90, p=2, weights=uniform;, score=0.328 total time=   0.0s
[CV 5/5] END algorithm=brute, n_neighbors=90, p=2, weights=uniform;, score=0.250 total time=   0.0s
[CV 1/5] END algorithm=brute, n_neighbors=90, p=2, weights=distance;, score=0.261 total time=   0.0s
[CV 2/5] END algorithm=brute, n_neighbors=90, p=2, weights=distance;, score=0.272 total time=   

[CV 4/5] END algorithm=brute, n_neighbors=94, p=2, weights=uniform;, score=0.311 total time=   0.0s
[CV 5/5] END algorithm=brute, n_neighbors=94, p=2, weights=uniform;, score=0.267 total time=   0.0s
[CV 1/5] END algorithm=brute, n_neighbors=94, p=2, weights=distance;, score=0.261 total time=   0.0s
[CV 2/5] END algorithm=brute, n_neighbors=94, p=2, weights=distance;, score=0.289 total time=   0.0s
[CV 3/5] END algorithm=brute, n_neighbors=94, p=2, weights=distance;, score=0.306 total time=   0.0s
[CV 4/5] END algorithm=brute, n_neighbors=94, p=2, weights=distance;, score=0.339 total time=   0.0s
[CV 5/5] END algorithm=brute, n_neighbors=94, p=2, weights=distance;, score=0.333 total time=   0.0s
[CV 1/5] END algorithm=brute, n_neighbors=95, p=1, weights=uniform;, score=0.250 total time=   0.0s
[CV 2/5] END algorithm=brute, n_neighbors=95, p=1, weights=uniform;, score=0.289 total time=   0.0s
[CV 3/5] END algorithm=brute, n_neighbors=95, p=1, weights=uniform;, score=0.272 total time=   

[CV 1/5] END algorithm=brute, n_neighbors=99, p=1, weights=uniform;, score=0.272 total time=   0.0s
[CV 2/5] END algorithm=brute, n_neighbors=99, p=1, weights=uniform;, score=0.272 total time=   0.0s
[CV 3/5] END algorithm=brute, n_neighbors=99, p=1, weights=uniform;, score=0.289 total time=   0.0s
[CV 4/5] END algorithm=brute, n_neighbors=99, p=1, weights=uniform;, score=0.356 total time=   0.0s
[CV 5/5] END algorithm=brute, n_neighbors=99, p=1, weights=uniform;, score=0.283 total time=   0.0s
[CV 1/5] END algorithm=brute, n_neighbors=99, p=1, weights=distance;, score=0.272 total time=   0.0s
[CV 2/5] END algorithm=brute, n_neighbors=99, p=1, weights=distance;, score=0.317 total time=   0.0s
[CV 3/5] END algorithm=brute, n_neighbors=99, p=1, weights=distance;, score=0.306 total time=   0.0s
[CV 4/5] END algorithm=brute, n_neighbors=99, p=1, weights=distance;, score=0.361 total time=   0.0s
[CV 5/5] END algorithm=brute, n_neighbors=99, p=1, weights=distance;, score=0.322 total time=   

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30, ...],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             verbose=3)

In [16]:
grid_predictions = grid.predict(x_test)
print("Best param:",grid.best_params_)
print("Best estimator:",grid.best_estimator_)
print("Classification Report:",classification_report(y_test, grid_predictions))

Best param: {'algorithm': 'auto', 'n_neighbors': 34, 'p': 1, 'weights': 'distance'}
Best estimator: KNeighborsClassifier(n_neighbors=34, p=1, weights='distance')
Classification Report:               precision    recall  f1-score   support

           0       0.33      0.29      0.31         7
           1       0.57      0.67      0.62        12
           2       0.11      0.14      0.12         7
           3       0.22      0.22      0.22         9
           4       0.33      0.08      0.12        13
           5       0.67      0.22      0.33         9
           6       0.37      0.70      0.48        10
           7       0.33      0.38      0.36        13
           8       0.22      0.33      0.27        12
           9       0.00      0.00      0.00         8

    accuracy                           0.32       100
   macro avg       0.32      0.30      0.28       100
weighted avg       0.33      0.32      0.30       100

